In [1]:
import requests
import json
import pandas as pd

Для начала укажем сайт, к которому будем подключаться по апи, чтобы геокодировать адреса, и уникальный ключ:

In [2]:
URL = "https://geocode.search.hereapi.com/v1/geocode"

<u/>ВАЖНО:</u>
Ключ требуется перезапрашивать каждый раз!

In [3]:
api_key = input("Введите api-key и нажмите ENTER ")

Введите api-key и нажмите ENTER zmyoRB6C6sOJgj7i_cxqk70njziydahcFOTWejKYSRI


Напишем функцию, где будем геокодировать адреса при помощи HERE. На вход будет подаваться датафрейм с запросами (Столбец с адресом должен называться "Запрос"), на выходе - новый датафрейм с ответом от HERE:

In [4]:
def using_here(df):

    # Создадим пустой датафрейм, который заполним в конце и вернем как результат выполненной функции:
    answer = pd.DataFrame()
    
    # Теперь перебираем запросы и складываем результат в списки
    for i in range(len(df)):
        request = df['Запрос'][i]
        PARAMS = {'apikey':api_key,'q':request} 
        
        # при помощи библиотеки requests отправляем параметры:
        r = requests.get(url = URL, params = PARAMS) 
        
        # и получаем ответ в формате .json, который будем парсить:
        data = r.json()
        
        # не всегда находится нужный адрес, поэтому предусмотрим этот момент и заполним пропусками пустой ответ        
        if len(data['items']) > 0:
            df1 = pd.json_normalize(data['items'][0])
            df1.insert(0, "request", request)
            answer = pd.concat([answer, df1])
        else:
            row_list = []
            row_list.append(['NaN' for i in range(29)])
            df1 = pd.DataFrame(row_list)   
            df1.columns=['title', 'id', 'resultType', 'houseNumberType', 'access',
                           'houseNumberFallback', 'address.label', 'address.countryCode',
                           'address.countryName', 'address.state', 'address.county',
                           'address.city', 'address.district', 'address.street',
                           'address.postalCode', 'address.houseNumber', 'position.lat',
                           'position.lng', 'mapView.west', 'mapView.south', 'mapView.east',
                           'mapView.north', 'scoring.queryScore', 'scoring.fieldScore.county',
                           'scoring.fieldScore.city', 'scoring.fieldScore.streets',
                           'scoring.fieldScore.houseNumber', 'scoring.fieldScore.postalCode',
                           'localityType']
            df1.insert(0, "request", request)
            answer = pd.concat([answer, df1])
    
    # Переименуем столбцы под свои нужды и оставим только те, что нам нужны
    new_answer = pd.DataFrame()
    
    new_answer['Запрос'] = answer['request']
    new_answer['Адрес'] = answer['address.label']
    new_answer['Широта'] = answer['position.lat']
    new_answer['Долгота'] = answer['position.lng']
    new_answer['Страна'] = answer['address.countryName']
    new_answer['Округ'] = answer['address.state']
    new_answer['Субъект'] = answer['address.county']
    new_answer['Город'] = answer['address.city']
    new_answer['Населенный пункт'] = answer['address.district']
    new_answer['Улица'] = answer['address.street']
    new_answer['Дом'] = answer['address.houseNumber']
    new_answer['Индекс'] = answer['address.postalCode']
    new_answer['Статус запроса'] = answer['resultType']
    
    return new_answer       # Возвращаем датафрейм   

Открываем наш файл со столбцом <u/>"Запрос":</u>:

In [5]:
dataframe = pd.read_excel('Алексинский и Воскресенский кластеры.xlsx')

Вызываем функцию:

In [6]:
%%time
newdata = using_here(dataframe)

Wall time: 38.6 s


Проверяем, что получилось:

In [7]:
newdata[:10]

,Запрос,Адрес,Широта,Долгота,Страна,Округ,Субъект,Город,Населенный пункт,Улица,Дом,Индекс,Статус запроса
0,"141654, обл Московская, г Клин, д Аксениха, д. 1","Аксениха 3, Клин, Россия, 141654",56.17528,36.40266,Россия,Центральный федеральный округ,Московская область,Клин,Аксениха,Аксениха,3,141654,houseNumber
0,"141654, обл Московская, г Клин, д Аксениха, д. 2","Аксениха 2, Клин, Россия, 141654",56.17555,36.4027,Россия,Центральный федеральный округ,Московская область,Клин,Аксениха,Аксениха,2,141654,houseNumber
0,"141654, обл Московская, г Клин, д Аксениха, д. 12","Аксениха 10, Клин, Россия, 141654",56.1756,36.40617,Россия,Центральный федеральный округ,Московская область,Клин,Аксениха,Аксениха,10,141654,houseNumber
0,"141654, обл Московская, г Клин, д Аксениха, д. 16","Аксениха 10, Клин, Россия, 141654",56.1756,36.40617,Россия,Центральный федеральный округ,Московская область,Клин,Аксениха,Аксениха,10,141654,houseNumber
0,"141544, обл Московская, г Солнечногорск, д Бла...","Благовещенка 9, Солнечногорск, Россия, 141544",55.90955,37.23075,Россия,Центральный федеральный округ,Московская область,Солнечногорск,Благовещенка,Благовещенка,9,141544,houseNumber
0,"обл Московская, г Клин, д. 181","Клин, Центральный федеральный округ, Россия",56.33655,36.71933,Россия,Центральный федеральный округ,Московская область,Клин,NaN,NaN,NaN,141601,locality
0,"141607, обл Московская, г Клин, ул Миши Балаки...","улица Миши Балакирева 18/67, Клин, Россия, 141607",56.32512,36.71904,Россия,Центральный федеральный округ,Московская область,Клин,Клин,улица Миши Балакирева,18/67,141607,houseNumber
0,"141605, обл Московская, г Клин, ул Центральная...","Центральная улица 103, Клин, Россия, 141605",56.36992,36.70453,Россия,Центральный федеральный округ,Московская область,Клин,Клин,Центральная улица,103,141605,houseNumber
0,"141605, обл Московская, г Клин, ул Инженерная,...","Инженерная улица 203, Клин, Россия, 141605",56.37412,36.70578,Россия,Центральный федеральный округ,Московская область,Клин,Клин,Инженерная улица,203,141605,houseNumber
0,"141605, обл Московская, г Клин, ул Инженерная,...","Инженерная улица 205, Клин, Россия, 141605",56.37336,36.70539,Россия,Центральный федеральный округ,Московская область,Клин,Клин,Инженерная улица,205,141605,houseNumber


Заменяем пропуски на пустые значения:

In [8]:
import numpy as np
newdata = newdata.replace('NaN', '', regex=True)

И сохраняем в эксель:

In [9]:
newdata.to_excel('Геокодирование областей.xlsx', index=False)